In [1]:
from sklearn.model_selection import train_test_split, KFold
from scipy.io import loadmat
import numpy as np
from models import *

# set random seed
np.random.seed(42)

dataset = loadmat('coil20.mat')
label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
data = dataset['X']
n_class = len(np.unique(label))

# train-test-split
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)
kf = KFold(n_splits=10, shuffle=True)
lams = [2**i for i in range(-6, 13, 2)] # regularization parameter, lambda

In [2]:
def cross_val_acc(data, label, kf, model_class, lam=None, n_layer=1, activation='sigmoid'):
    n_node = 100 # num of nodes in hidden layer
    w_range = [-1, 1] # range of random weights
    b_range = [0, 1] # range of random biases
    val_acc = []

    # print('Lambda: ', lam)
    for i, kf_values in enumerate(kf.split(data, label)):
    #     print(f'train: {train_index}, val: {val_index}')
        # print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = data[train_index], data[val_index]
        y_val_train, y_val_test = label[train_index], label[val_index]
        if model_class in [RVFL, DeepRVFL, EnsembleDeepRVFL]:
            model = model_class(n_node, lam, w_range, b_range, n_layer, activation=activation)
        elif model_class in [BRVFL, BDeepRVFL, BEnsembleDeepRVFL]:
            model = model_class(n_node, w_range, b_range, n_layer, activation=activation)
        model.train(X_val_train, y_val_train, n_class)
        acc = model.eval(X_val_test, y_val_test)
        # print(f'Validation accuracy: {acc}')
        val_acc.append(acc)

    mean_acc = np.mean(val_acc)
    # print(f'Model accuracy: {mean_acc}\n')
    return model, mean_acc

In [12]:
# RVFL
acc = []
for lam in lams:
    _, model_accuracy = cross_val_acc(data, label, kf, RVFL, lam)
    acc.append(model_accuracy)

max_acc = np.max(acc)
opt_lam = lams[acc.index(max_acc)]
print('Accuracy: ', max_acc)
print('Lambda: ', opt_lam)

Accuracy:  0.9673611111111111
Lambda:  64


In [22]:
# dRVFL
acc = []
for lam in lams:
    _, model_accuracy = cross_val_acc(data, label, kf, DeepRVFL, lam, n_layer=5)
    acc.append(model_accuracy)

max_acc = np.max(acc)
opt_lam = lams[acc.index(max_acc)]
print('Accuracy: ', max_acc)
print('Lambda: ', opt_lam)

Accuracy:  0.9715277777777779
Lambda:  16


In [24]:
# edRVFL
acc = []
for lam in lams:
    _, model_accuracy = cross_val_acc(data, label, kf, EnsembleDeepRVFL, lam, n_layer=5)
    acc.append(model_accuracy)

max_acc = np.max(acc)
opt_lam = lams[acc.index(max_acc)]
print('Accuracy: ', max_acc)
print('Lambda: ', opt_lam)

Accuracy:  0.9729166666666667
Lambda:  64


In [6]:
# BRVFL
acc = []
model, model_accuracy = cross_val_acc(data, label, kf, BRVFL)
acc.append(model_accuracy)

print('Accuracy: ', model_accuracy)
print('Precision: ', model.prec)
print('Variance: ', model.var)


Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  16  iterations



Convergence after  15  iterations



Convergence after  17  iterations



Convergence after  16  iterations
Accuracy:  0.9423611111111112
Precision:  73.62270211352329
Variance:  0.003308063420566107


In [3]:
# BdRVFL

acc = []
model, model_accuracy = cross_val_acc(data, label, kf, BDeepRVFL, n_layer=5)
acc.append(model_accuracy)

print('Accuracy: ', model_accuracy)
print('Precision: ', model.prec)
print('Variance: ', model.var)


Convergence after  5  iterations



Convergence after  5  iterations



Convergence after  5  iterations



Convergence after  4  iterations



Convergence after  5  iterations



Convergence after  4  iterations



Convergence after  4  iterations



Convergence after  5  iterations



Convergence after  5  iterations



Convergence after  4  iterations
Accuracy:  0.9381944444444444
Precision:  67.44775611126619
Variance:  6.141261880691838e-06


In [2]:
# BedRVFL

acc = []
model, model_accuracy = cross_val_acc(data, label, kf, BEnsembleDeepRVFL, n_layer=5)
acc.append(model_accuracy)

print('Accuracy: ', model_accuracy)
print('Precision: ', model.prec)
print('Variance: ', model.var)


Convergence after  17  iterations



Convergence after  16  iterations



Convergence after  17  iterations



Convergence after  17  iterations



Convergence after  17  iterations

Test accuracy: 0.9201388888888888
